## Warning!!! Avert your eyes and proceed to notebooks folder. 

This notebook is just for the developer's reference. It is a stream of thought multivariate analysis so I could design the stats module. Wanted to add it to scripts so that I could reference it if need be. But it is not recommended to use this

In [ ]:
from bokeh.plotting import figure, output_file, show
import os 
from bokeh.io import output_notebook
import numpy as np
import pandas as pd
from sqlalchemy import *
output_notebook()
import pickle as pk
from itertools import combinations as comb

from pydoc import help  # can type in the python console `help(name of function)` to get the documentation
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy import stats
from IPython.display import display, HTML

import colcol as c
# figures inline in notebook
%matplotlib inline

np.set_printoptions(suppress=True)

DISPLAY_MAX_ROWS = 20  # number of max rows to print for a DataFrame
pd.set_option('display.max_rows', DISPLAY_MAX_ROWS)

In [ ]:
fs = c.print_filters('wfirst')

In [ ]:
dirr = os.getenv('ALBEDO_DB')
engine = create_engine('sqlite:///'+dirr+'AlbedoModels_2015.db')
header = pd.read_sql_table('header',engine)
et= pk.load(open('/Users/natashabatalha/Documents/colorcolor/notebooks/wfirst_colors_dataframe.pk','rb'))#pk.load(open('/Volumes/MyPassportforMac/WFIRST/FluxDataFrameWFIRST.pk','rb'))

In [ ]:
et = et.reset_index(drop=True)
everything=et.dropna()[~et.dropna().isin([np.inf, -np.inf])]
everything = everything.dropna()[(everything['cloud']==0) & (everything['phase']==90)]
yofinterest = 'metallicity'

In [ ]:
#everything
fcomb = [i[0]+i[1] for i in comb(fs,2)] +list(fs)#independent varianbles 
#fcomb = [i for i in fs]
#fcomb += [ '506', '575', '661', '721', '883', '940']

In [ ]:
#corrmat = everything.loc[:,fcomb].corr()
corrmat = everything.loc[(everything['cloud']==0)].corr()

In [ ]:
fig, ax = plt.subplots(figsize=(25,10)) 
sns.heatmap(corrmat, vmax=1, square=False, linewidths=.5, ax=ax).xaxis.tick_top()
#.abs()

# Correlation Analysis

In [ ]:
def mosthighlycorrelated(mydataframe, numtoreport):
    # find the correlations
    cormatrix = mydataframe.corr()
    # set the correlations on the diagonal or lower triangle to zero,
    # so they will not be reported as the highest ones:
    cormatrix *= np.tri(*cormatrix.values.shape, k=-1).T
    # find the top n correlations
    cormatrix = cormatrix.stack()
    cormatrix = cormatrix.reindex(cormatrix.abs().sort_values(ascending=False).index).reset_index()
    # assign human-friendly names
    cormatrix.columns = ["FirstVariable", "SecondVariable", "Correlation"]
    return cormatrix.head(numtoreport)
x = mosthighlycorrelated(everything.loc[:,fcomb].corr(), 10)

# PCA Analysis

In [ ]:
standardisedX = scale(everything.loc[:,fcomb])
standardisedX = pd.DataFrame(standardisedX, index=everything.loc[:,fcomb].index, columns=everything.loc[:,fcomb].columns)

In [ ]:
pca = PCA().fit(standardisedX)

In [ ]:
def pca_summary(pca, standardised_data, out=True):
    names = ["PC"+str(i) for i in range(1, len(pca.explained_variance_ratio_)+1)]
    
    a = list(np.std(pca.transform(standardised_data), axis=0))
    b = list(pca.explained_variance_ratio_)
    c = [np.sum(pca.explained_variance_ratio_[:i]) for i in range(1, len(pca.explained_variance_ratio_)+1)]
    columns = pd.MultiIndex.from_tuples([("sdev", "Standard deviation"), ("varprop", "Proportion of Variance"), ("cumprop", "Cumulative Proportion")])
    summary = pd.DataFrame(np.c_[a, b, c], index=names, columns=columns)
    if out:
        print("Importance of components:")
        display(summary)
    return summary

In [ ]:
summary = pca_summary(pca, standardisedX)

In [ ]:
np.sum(summary.sdev**2)

In [ ]:
def screeplot(pca, standardised_values):
    y = np.std(pca.transform(standardised_values), axis=0)**2
    x = np.arange(len(y)) + 1
    plt.plot(x, y, "o-")
    plt.xticks(x, ["Comp."+str(i) for i in x], rotation=60)
    plt.ylabel("Variance")
    plt.show()

screeplot(pca, standardisedX)

In [ ]:
summary.sdev**2

In [ ]:
for i, j in zip(pca.components_[0], fcomb): print( i,j)

In [ ]:
def calcpc(variables, loadings):
    # find the number of samples in the data set and the number of variables
    numsamples, numvariables = variables.shape
    # make a vector to store the component
    pc = np.zeros(numsamples)
    # calculate the value of the component for each sample
    for i in range(numsamples):
        valuei = 0
        for j in range(numvariables):
            valueij = variables.iloc[i, j]
            loadingj = loadings[j]
            valuei = valuei + (valueij * loadingj)
        pc[i] = valuei
    return pc

In [ ]:
calcpc(standardisedX, pca.components_[0])


In [ ]:
def pca_scatter(pca, standardised_values, classifs):
    foo = pca.transform(standardised_values)
    bar = pd.DataFrame(list(zip(foo[:, 0], foo[:, 1], classifs)), columns=["PC1", "PC2", "Class"])
    sns.lmplot("PC1", "PC2", bar, hue="Class", fit_reg=False)

In [ ]:
X = everything.loc[:,fcomb]
y =  everything[yofinterest].astype(str)
#newy = []
#for i in y: 
#    if float(i) >= 1.7: newy+=['3']
#    elif float(i) <= 0.5: newy+=['1']
#    else: newy +=['2']


In [ ]:
pca_scatter(pca, standardisedX, y)

# LDA Analysis

In [ ]:
#classifier so needs to be string.. 
lda = LinearDiscriminantAnalysis().fit(X,y)

In [ ]:
def pretty_scalings(lda, X, out=False):
    ret = pd.DataFrame(lda.scalings_, index=X.columns, columns=["LD"+str(i+1) for i in range(lda.scalings_.shape[1])])
    if out:
        print("Coefficients of linear discriminants:")
        display(ret)
    return ret

pretty_scalings_ = pretty_scalings(lda, X, out=True)

In [ ]:
pretty_scalings_.LD1

In [ ]:
def calcWithinGroupsVariance(variable, groupvariable):
    # find out how many values the group variable can take
    levels = sorted(set(groupvariable))
    numlevels = len(levels)
    # get the mean and standard deviation for each group:
    numtotal = 0
    denomtotal = 0
    for leveli in levels:
        levelidata = variable[groupvariable==leveli]
        levelilength = len(levelidata)
        # get the standard deviation for group i:
        sdi = np.std(levelidata)
        numi = (levelilength)*sdi**2
        denomi = levelilength
        numtotal = numtotal + numi
        denomtotal = denomtotal + denomi
    # calculate the within-groups variance
    Vw = numtotal / (denomtotal - numlevels)
    return Vw
def calcBetweenGroupsVariance(variable, groupvariable):
    # find out how many values the group variable can take
    levels = sorted(set((groupvariable)))
    numlevels = len(levels)
    # calculate the overall grand mean:
    grandmean = np.mean(variable)
    # get the mean and standard deviation for each group:
    numtotal = 0
    denomtotal = 0
    for leveli in levels:
        levelidata = variable[groupvariable==leveli]
        levelilength = len(levelidata)
        # get the mean and standard deviation for group i:
        meani = np.mean(levelidata)
        sdi = np.std(levelidata)
        numi = levelilength * ((meani - grandmean)**2)
        denomi = levelilength
        numtotal = numtotal + numi
        denomtotal = denomtotal + denomi
    # calculate the between-groups variance
    Vb = numtotal / (numlevels - 1)
    return(Vb)
def calclda(variables, loadings):
    # find the number of samples in the data set and the number of variables
    numsamples, numvariables = variables.shape
    # make a vector to store the discriminant function
    ld = np.zeros(numsamples)
    # calculate the value of the discriminant function for each sample
    for i in range(numsamples):
        valuei = 0
        for j in range(numvariables):
            valueij = variables.iloc[i, j]
            loadingj = loadings[j]
            valuei = valuei + (valueij * loadingj)
        ld[i] = valuei
    # standardise the discriminant function so that its mean value is 0:
    ld = scale(ld, with_std=False)
    return ld
def groupStandardise(variables, groupvariable):
    # find the number of samples in the data set and the number of variables
    numsamples, numvariables = variables.shape
    # find the variable names
    variablenames = variables.columns
    # calculate the group-standardised version of each variable
    variables_new = pd.DataFrame()
    for i in range(numvariables):
        variable_name = variablenames[i]
        variablei = variables[variable_name]
        variablei_Vw = calcWithinGroupsVariance(variablei, groupvariable)
        variablei_mean = np.mean(variablei)
        variablei_new = (variablei - variablei_mean)/(np.sqrt(variablei_Vw))
        variables_new[variable_name] = variablei_new
    return variables_new
def calcSeparations(variables, groupvariable):
    # calculate the separation for each variable
    for variablename in variables:
        variablei = variables[variablename]
        Vw = calcWithinGroupsVariance(variablei, groupvariable)
        Vb = calcBetweenGroupsVariance(variablei, groupvariable)
        sep = Vb/Vw
        print("variable", variablename, "Vw=", Vw, "Vb=", Vb, "separation=", sep)

In [ ]:
print(calclda(X, lda.scalings_[:, 0]))
print(lda.fit_transform(X, y)[:, 0])

In [ ]:
groupstandardisedX = groupStandardise(X, y)

In [ ]:
y

In [ ]:
lda2 = LinearDiscriminantAnalysis().fit(groupstandardisedX, y)
pretty_scalings(lda2, groupstandardisedX)

In [ ]:
lda.fit_transform(X, y)[:, 0]

In [ ]:
lda2.fit_transform(groupstandardisedX, y)[:, 0]

In [ ]:
def rpredict(lda, X, y, out=False):
    ret = {"class": lda.predict(X),
           "posterior": pd.DataFrame(lda.predict_proba(X), columns=lda.classes_)}
    ret["x"] = pd.DataFrame(lda.fit_transform(X, y))
    ret["x"].columns = ["LD"+str(i+1) for i in range(ret["x"].shape[1])]
    #if out:
    #    print("class")
    #    print(ret['class'])# : print(i)
    #    print()
    #    print("posterior")
    #    print(ret["posterior"])
    #    print()
    #    print("x")
    #    print(ret["x"])
    return ret

lda_values = rpredict(lda2, standardisedX.reset_index(drop=True), y.reset_index(drop=True), True)

In [ ]:
calcSeparations(lda_values["x"], y.reset_index(drop=True))

In [ ]:
def ldahist(data, g, sep=False):
    xmin = np.trunc(np.min(data)) - 1
    xmax = np.trunc(np.max(data)) + 1
    ncol = len(set(g))
    print (ncol)
    binwidth = 0.5
    bins=np.arange(xmin, xmax + binwidth, binwidth)
    if sep:
        fig, axl = plt.subplots(ncol, 1, sharey=True, sharex=True)
    else:
        fig, axl = plt.subplots(1, 1, sharey=True, sharex=True)
        axl = [axl]*ncol
    for ax, (group, gdata) in zip(axl, data.groupby(g)):
        sns.distplot(gdata.values, bins, ax=ax, label="group "+str(group))
        ax.set_xlim([xmin, xmax])
        if sep:
            ax.set_xlabel("group"+str(group))
        else:
            ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.tight_layout()

In [ ]:
ldahist(lda_values["x"].LD1, y.reset_index(drop=True))

In [ ]:
ldahist(lda_values["x"].LD2, y.reset_index(drop=True))

In [ ]:
sns.lmplot("LD1", "LD2", lda_values["x"].join(y.reset_index(drop=True)), hue=yofinterest, fit_reg=False);

In [ ]:
def printMeanAndSdByGroup(variables, groupvariable):
    data_groupby = variables.groupby(groupvariable)
    print("## Means:")
    display(data_groupby.apply(np.mean))
    print("\n## Standard deviations:")
    display(data_groupby.apply(np.std))
    print("\n## Sample sizes:")
    display(pd.DataFrame(data_groupby.apply(len)))
    return data_groupby.apply(np.mean)
means = printMeanAndSdByGroup(lda_values["x"], y.reset_index(drop=True))

In [ ]:
import sklearn.metrics as metrics

In [ ]:
cutoffs = []
for i in range(len(means['LD1'])): 
    cutoffs +=[np.mean(means['LD1'][i:i+2])]
cutoffs = cutoffs[:-1]
print(cutoffs)

In [ ]:
def lda_classify(v, levels, cutoffpoints):
    for level, cutoff in zip(reversed(levels), reversed(cutoffpoints)):
        if v > cutoff: return level
    return levels[0]
    
y_pred = lda_values["x"].iloc[:, 0].apply(lda_classify, args=(lda.classes_, cutoffs,)).values
y_true = y.reset_index(drop=True)

In [ ]:
def webprint_confusion_matrix(confusion_matrix, classes_names):
    display(pd.DataFrame(confusion_matrix, index=["Is group "+i for i in classes_names], columns=["Allocated to group "+i for i in classes_names]))

def plot_confusion_matrix(cm, target_names, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(target_names))
    plt.xticks(tick_marks, target_names, rotation=45)
    plt.yticks(tick_marks, target_names)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

print(metrics.classification_report(y_true, y_pred))
cm = metrics.confusion_matrix(y_true, y_pred)
webprint_confusion_matrix(cm, lda.classes_)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
plot_confusion_matrix(cm_normalized, lda.classes_, title='Normalized confusion matrix')

In [ ]:
everything.loc[:,fcomb].values.shape